In [1]:
import pandas as pd
import numpy as np
import random
import re
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from mlxtend.frequent_patterns import apriori, association_rules

# Preparing The Data

In [2]:
data= pd.read_csv('../ml-25m/movies.csv')

In [3]:
ratings_data=pd.read_csv('../ml-25m/ratings.csv',sep=',')

In [4]:
movie_data_ratings_data=data.merge(ratings_data,on = 'movieId', how='inner')

In [5]:
# Group movies_data_ratings_data by userId and create a new column called watched_movies which contains name of all movies watched by the user
movie_data_ratings_data_grouped_by_userId = movie_data_ratings_data.groupby("userId")
movie_data_ratings_data_grouped_by_userId = movie_data_ratings_data_grouped_by_userId["title"].apply(list).reset_index(name="watched_movies")
movie_data_ratings_data_grouped_by_userId.head()
# Create a new dataframe which contains encoded watched_movies column
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
movie_data_ratings_data_grouped_by_userId = movie_data_ratings_data_grouped_by_userId.iloc[:1000] # to reduce the size of dataframe to 1000 rows
movie_data_ratings_data_encoded = movie_data_ratings_data_grouped_by_userId.join(pd.DataFrame(mlb.fit_transform(movie_data_ratings_data_grouped_by_userId.pop('watched_movies')),
                          columns=mlb.classes_,
                          index=movie_data_ratings_data_grouped_by_userId.index))

# Apriori


In [6]:
movie_data_ratings_data_encoded.head()

,userId,$5 a Day (2008),'71 (2014),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),(Dis)Honesty: The Truth About Lies (2015),...,¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),В движении (2002),Кентервильское привидение (1970),Когда зажигаются ёлки (1950),Ландыш серебристый (2000),Обезьянки и грабители (1985),Он вам не Димон (2017),Приключения Домовёнка (1986),Принцесса и Людоед (1977)
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
min_support = 0.1

In [8]:
sampled_data = movie_data_ratings_data_grouped_by_userId.sample(frac=0.25, random_state=42)
frequent_itemsets = apriori(movie_data_ratings_data_encoded.drop(columns=["userId"]), min_support=min_support, use_colnames=True)

/home/mustafa/anaconda3/envs/mining_project/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [9]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

In [10]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(2001: A Space Odyssey (1968)),(Alien (1979)),0.152,0.192,0.102,0.671053,3.495066,0.072816,2.456320,0.841842
1,(Alien (1979)),(2001: A Space Odyssey (1968)),0.192,0.152,0.102,0.531250,3.495066,0.072816,1.809067,0.883518
2,(2001: A Space Odyssey (1968)),(Back to the Future (1985)),0.152,0.292,0.105,0.690789,2.365717,0.060616,2.289702,0.680773
3,(2001: A Space Odyssey (1968)),(Blade Runner (1982)),0.152,0.214,0.107,0.703947,3.289474,0.074472,2.654933,0.820755
4,(Blade Runner (1982)),(2001: A Space Odyssey (1968)),0.214,0.152,0.107,0.500000,3.289474,0.074472,1.696000,0.885496


In [11]:
rules["confidence"].min()

0.5

In [12]:
random_sample = rules.sample()

In [13]:
print(random_sample["antecedents"])

144837    (Dances with Wolves (1990), Forrest Gump (1994))
Name: antecedents, dtype: object


In [14]:
random_sample

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
144837,"(Dances with Wolves (1990), Forrest Gump (1994))","(Silence of the Lambs, The (1991), Jurassic Pa...",0.2,0.166,0.11,0.55,3.313253,0.0768,1.853333,0.872727


In [15]:
# Generate frequent itemsets for different sizes (e.g., 3, 4, 5, 6)
frequent_itemsets_3 = apriori(movie_data_ratings_data_encoded.drop(columns=["userId"]), min_support=min_support, use_colnames=True, max_len=3)
rules_3 = association_rules(frequent_itemsets_3, metric="confidence", min_threshold=0.7)

/home/mustafa/anaconda3/envs/mining_project/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [16]:
frequent_itemsets_4 = apriori(movie_data_ratings_data_encoded.drop(columns=["userId"]), min_support=min_support, use_colnames=True, max_len=4)
rules_4 = association_rules(frequent_itemsets_4, metric="confidence", min_threshold=0.7)

/home/mustafa/anaconda3/envs/mining_project/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [26]:
frequent_itemsets_5 = apriori(movie_data_ratings_data_encoded.drop(columns=["userId"]), min_support=min_support, use_colnames=True, max_len=5)
rules_5 = association_rules(frequent_itemsets_5, metric="confidence", min_threshold=0.5)

/home/mustafa/anaconda3/envs/mining_project/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [27]:
frequent_itemsets_6 = apriori(movie_data_ratings_data_encoded.drop(columns=["userId"]), min_support=min_support, use_colnames=True, max_len=6)
rules_6 = association_rules(frequent_itemsets_6, metric="confidence", min_threshold=0.5)

/home/mustafa/anaconda3/envs/mining_project/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [19]:
rules_3.sample()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
11755,"(Lord of the Rings: The Return of the King, Th...",(Pulp Fiction (1994)),0.135,0.477,0.101,0.748148,1.568445,0.036605,2.076618,0.418989


In [20]:
rules_4.sample()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
50768,"(Usual Suspects, The (1995), Star Wars: Episod...",(Jurassic Park (1993)),0.134,0.4,0.104,0.776119,1.940299,0.0504,2.68,0.559602


In [28]:
rules_5.sample()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
98104,"(Silence of the Lambs, The (1991), Good Will H...","(Sixth Sense, The (1999), Pulp Fiction (1994))",0.162,0.192,0.109,0.67284,3.504372,0.077896,2.469736,0.852795


In [22]:
rules_6.sample()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
20080,"(Star Wars: Episode IV - A New Hope (1977), Us...","(Silence of the Lambs, The (1991))",0.114,0.441,0.102,0.894737,2.028882,0.051726,5.3105,0.572368


In [34]:
# Find consquents with more than 1 item
rules_6[rules_6["consequents"].apply(lambda x: len(x) > 4)]


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
221236,(Aliens (1986)),(Star Wars: Episode VI - Return of the Jedi (1...,0.177,0.114,0.100,0.564972,4.955893,0.079822,2.036649,0.969891
221237,(Alien (1979)),(Star Wars: Episode VI - Return of the Jedi (1...,0.192,0.111,0.100,0.520833,4.692192,0.078688,1.855304,0.973861
221290,(Alien (1979)),(Star Wars: Episode VI - Return of the Jedi (1...,0.192,0.127,0.101,0.526042,4.142060,0.076616,1.841934,0.938830
221340,(Alien (1979)),(Star Wars: Episode VI - Return of the Jedi (1...,0.192,0.132,0.101,0.526042,3.985164,0.075656,1.831385,0.927066
221390,(Aliens (1986)),(Star Wars: Episode VI - Return of the Jedi (1...,0.177,0.132,0.101,0.570621,4.322890,0.077636,2.021526,0.933989
224925,(Indiana Jones and the Last Crusade (1989)),(Star Wars: Episode VI - Return of the Jedi (1...,0.219,0.135,0.110,0.502283,3.720616,0.080435,1.737936,0.936271
226254,(Star Wars: Episode I - The Phantom Menace (19...,(Star Wars: Episode VI - Return of the Jedi (1...,0.193,0.139,0.103,0.533679,3.839416,0.076173,1.846367,0.916411
227180,"(Terminator, The (1984))",(Star Wars: Episode VI - Return of the Jedi (1...,0.210,0.127,0.106,0.504762,3.974503,0.079330,1.762788,0.947337
237473,(Die Hard: With a Vengeance (1995)),"(Pulp Fiction (1994), Fugitive, The (1993), Ju...",0.219,0.148,0.110,0.502283,3.393805,0.077588,1.711817,0.903131
237658,(Die Hard: With a Vengeance (1995)),"(Fugitive, The (1993), Jurassic Park (1993), F...",0.219,0.142,0.112,0.511416,3.601518,0.080902,1.756093,0.924890
